In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [6]:
mp_holistic = mp.solutions.holistic    # holistic model
mp_drawing = mp.solutions.drawing_utils    # drawing utilities

In [8]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
cap = cv2.VideoCapture(0)    # access our webcam
while cap.isOpened():    # read videofeed from webcam, loop through all frames in camera
    # read feed
    ret, frame = cap.read()    # grab current frame from our camera at a point in time
    # show to screen
    cv2.imshow("OpenCV Feed", frame)
    # breaking gracefully
    if cv2.waitKey(10) & 0xFF==ord('q'):    # wait for key to be pressed inside our frame and if key==q then break out
        break
cap.release()    # release our webcam
cv2.destroyAllWindows()    # close our frame